In [1]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.companies


In [4]:
import pandas as pd
df = pd.DataFrame(db.comp.find())
df.head()



,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8b,None,[],Zoho ManageEngine,,,enterprise,[],2007-05-25 19:24:22,http://www.crunchbase.com/company/adventnet,...,925-924-9500,[],"[{'title': 'DHFH', 'is_past': True, 'provider'...","[{'is_past': True, 'title': 'CEO and Co-Founde...","[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,manageengine,Wed Oct 31 18:26:09 UTC 2012,[]
1,52cdef7c4bab8bd675297d8a,"{'price_amount': 30000000, 'price_currency_cod...",[],,http://digitalquarters.net/feed/,http://digitalquarters.net/,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",2007-05-25 06:51:27,http://www.crunchbase.com/company/wetpaint,...,206.859.6300,"[{'name': 'Wikison Wetpaint', 'permalink': 'we...",[],"[{'is_past': False, 'title': 'Co-Founder and V...","[{'available_sizes': [[[150, 86], 'assets/imag...","wiki, seattle, elowitz, media-industry, media-...",$39.8M,BachelrWetpaint,Sun Dec 08 07:15:44 UTC 2013,[]
2,52cdef7c4bab8bd675297d8c,None,[],,http://blogs.zoho.com/feed,http://blogs.zoho.com/,software,"[{'competitor': {'name': 'Empressr', 'permalin...",Fri May 25 19:30:28 UTC 2007,http://www.crunchbase.com/company/zoho,...,1-888-204-3539,"[{'name': 'Zoho Office Suite', 'permalink': 'z...",[],"[{'is_past': False, 'title': 'CEO and Founder'...",[],"zoho, officesuite, spreadsheet, writer, projec...",$0,zoho,Wed Oct 30 00:07:05 UTC 2013,"[{'embed_code': '<object width=""430"" height=""2..."
3,52cdef7c4bab8bd675297d8d,"{'price_amount': 500000, 'price_currency_code'...","[{'price_amount': None, 'price_currency_code':...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",Fri May 25 20:03:23 UTC 2007,http://www.crunchbase.com/company/digg,...,(415) 436-9638,"[{'name': 'Digg', 'permalink': 'digg'}]","[{'title': 'Public Relations', 'is_past': True...","[{'is_past': False, 'title': 'CEO', 'person': ...","[{'available_sizes': [[[117, 150], 'assets/ima...","community, social, news, bookmark, digg, techn...",$45M,digg,Tue Nov 05 21:35:47 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl..."
4,52cdef7c4bab8bd675297d8f,None,[],None,http://feeds.feedburner.com/omnidrive,http://www.omnidrive.com/blog,network_hosting,"[{'competitor': {'name': 'Dropbox', 'permalink...",Sun May 27 03:25:32 UTC 2007,http://www.crunchbase.com/company/omnidrive,...,660-675-5052,"[{'name': 'Omnidrive', 'permalink': 'omnidrive'}]",[],"[{'is_past': True, 'title': 'Co-founder', 'per...",[],"storage, sharing, edit, online",$800k,Nomadesk,Tue Jul 02 22:48:04 UTC 2013,[]


In [5]:
df.columns

Index(['_id', 'acquisition', 'acquisitions', 'alias_list', 'blog_feed_url',
       'blog_url', 'category_code', 'competitions', 'created_at',
       'crunchbase_url', 'deadpooled_day', 'deadpooled_month',
       'deadpooled_url', 'deadpooled_year', 'description', 'email_address',
       'external_links', 'founded_day', 'founded_month', 'founded_year',
       'funding_rounds', 'homepage_url', 'image', 'investments', 'ipo',
       'milestones', 'name', 'number_of_employees', 'offices', 'overview',
       'partners', 'permalink', 'phone_number', 'products', 'providerships',
       'relationships', 'screenshots', 'tag_list', 'total_money_raised',
       'twitter_username', 'updated_at', 'video_embeds'],
      dtype='object')

In [7]:
df["description"].value_counts()

                                            2990
software solutions                            17
Search Engine Optimization                    14
Social Network                                11
fabless semiconductor company                 11
Computer Software                              9
Semiconductor Company                          8
Social Media Marketing                         7
Web Design                                     6
semiconductor company                          6
Online Media                                   6
Internet Marketing                             5
Online Marketing                               5
Healthcare Biotechnology                       5
Marketing and Advertising                      5
IT Services                                    5
Software and Services                          5
Web Design and Development                     5
software developer                             5
software company                               4
Biopharmaceutical   

In [9]:
df["founded_year"].value_counts()

2008.0    2475
2007.0    2277
2006.0    1423
2005.0     961
2004.0     752
2009.0     687
2003.0     576
1999.0     533
2000.0     521
2001.0     464
2002.0     460
1998.0     290
1996.0     216
1997.0     200
1995.0     145
1994.0      97
1993.0      81
2010.0      60
1992.0      58
1991.0      46
1989.0      46
1986.0      45
2011.0      43
1987.0      42
1990.0      41
1983.0      36
1985.0      34
1988.0      33
2012.0      30
1982.0      29
          ... 
1861.0       1
1848.0       1
1953.0       1
1903.0       1
1867.0       1
1885.0       1
1941.0       1
1936.0       1
1939.0       1
1925.0       1
1892.0       1
1843.0       1
1934.0       1
1931.0       1
1910.0       1
1924.0       1
1870.0       1
1835.0       1
1836.0       1
1949.0       1
1907.0       1
1900.0       1
1863.0       1
1883.0       1
1928.0       1
1839.0       1
1894.0       1
1914.0       1
1896.0       1
1857.0       1
Name: founded_year, Length: 137, dtype: int64

In [11]:
df["number_of_employees"].value_counts()

2.0         799
5.0         726
3.0         697
1.0         599
10.0        588
4.0         500
6.0         309
15.0        295
20.0        271
8.0         229
12.0        220
7.0         219
50.0        215
25.0        202
30.0        180
100.0       164
0.0         137
40.0        126
9.0         120
35.0         89
60.0         87
200.0        84
14.0         81
11.0         79
45.0         67
150.0        64
80.0         60
70.0         58
18.0         55
16.0         54
           ... 
176.0         1
208.0         1
4239.0        1
456.0         1
948.0         1
5200.0        1
388000.0      1
13000.0       1
715.0         1
422.0         1
5729.0        1
6400.0        1
216.0         1
14000.0       1
320000.0      1
92900.0       1
97.0          1
950.0         1
79183.0       1
1618.0        1
1112.0        1
13500.0       1
223.0         1
140000.0      1
7000.0        1
5299.0        1
5300.0        1
186000.0      1
227000.0      1
8100.0        1
Name: number_of_employee